<a href="https://colab.research.google.com/github/ChitranshBose/CSE508_Winter2023_A2_47/blob/main/IR_A2_47_ASSIGN2_Q2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [444]:
# importing necessary libraries
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')
import seaborn as sns
from sklearn.naive_bayes import MultinomialNB
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split 

from sklearn.preprocessing import  StandardScaler 
from sklearn import metrics as mt

import nltk
nltk.download('punkt')
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

import re

from sklearn.feature_extraction.text import CountVectorizer

from nltk.stem import PorterStemmer


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


**Preprocessing** 

In [445]:
# reading data and removing the unncessary column
data_train = pd.read_csv("BBC News Train.csv", on_bad_lines='skip', encoding="ISO-8859-1")
data_train.drop(['ArticleId'],inplace=True,axis=1)
data_train.head()


,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [446]:
# making a copy of dataset
data_train_temp = data_train.copy()
data_train_temp

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business
...,...,...
1485,double eviction from big brother model caprice...,entertainment
1486,dj double act revamp chart show dj duo jk and ...,entertainment
1487,weak dollar hits reuters revenues at media gro...,business
1488,apple ipod family expands market apple has exp...,tech


In [447]:
# lowering the case of the dataset
data_train['Category'] = data_train['Category'].str.lower()
data_train['Text'] = data_train['Text'].str.lower()
data_train.head()

,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [448]:
#function to remove punctuation
def removePunctuation(data):
  token = RegexpTokenizer(r"\w+")
  return token.tokenize(''.join(data))

In [449]:
data_train['Text'] = data_train['Text'].apply(removePunctuation)
data_train.head()

,Text,Category
0,"[worldcom, ex, boss, launches, defence, lawyer...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, in, 168m, payout, eighteen, fo...",business


In [450]:
# function to remove stop words
stopwords = stopwords.words('english')

def removeWords(data):
  
  output = []
  for word in data:
    if word not in stopwords:
      output.append(word)
  return output

In [451]:
data_train['Text'] = data_train['Text'].apply(removeWords)
data_train.head()

,Text,Category
0,"[worldcom, ex, boss, launches, defence, lawyer...",business
1,"[german, business, confidence, slides, german,...",business
2,"[bbc, poll, indicates, economic, gloom, citize...",business
3,"[lifestyle, governs, mobile, choice, faster, b...",tech
4,"[enron, bosses, 168m, payout, eighteen, former...",business


In [452]:
#function to lemmatize the data
def lemmatize(data):
  output = []
  lst = ['a','r','n','v']
  wordNetLemmatizer = WordNetLemmatizer()
  posTag = pos_tag(data)
  for word, tag in posTag:
    pos = tag[0].lower()

    if pos not in lst:
      pos = 'n'
    output.append(wordNetLemmatizer.lemmatize(word,pos))
  return output

In [453]:
data_train['Text'] = data_train['Text'].apply(lemmatize)
data_train.head()

,Text,Category
0,"[worldcom, ex, bos, launch, defence, lawyer, d...",business
1,"[german, business, confidence, slide, german, ...",business
2,"[bbc, poll, indicate, economic, gloom, citizen...",business
3,"[lifestyle, governs, mobile, choice, faster, w...",tech
4,"[enron, boss, 168m, payout, eighteen, former, ...",business


In [454]:
# function for applying stemming to the data
def stemming(data):
  output = []
  stem = PorterStemmer()
  for word in data:
    output.append(stem.stem(word))
  return output

In [455]:
categories = data_train['Category'].unique() #finding classes

term_freq = {}
class_freq = {}
num_class = len(categories)

In [456]:
# function to find the sentences based on the category they belong to
def get_cat(categories, data):
  
  cat_business = data[data['Category'] == 'business']
    
  cat_tech = data[data['Category'] == 'tech']
    
  cat_politics = data[data['Category'] == 'politics']

  cat_sport = data[data['Category'] == 'sport']
    
  cat_entertain = data[data['Category'] == 'entertainment']
    
  
  return cat_business, cat_tech, cat_politics, cat_sport, cat_entertain
      

In [457]:
cat_business, cat_tech, cat_politics, cat_sport, cat_entertain = get_cat(categories,data_train)


In [458]:
#function to calculate term frequency
def cal_tf(term_freq_data,data,cat_type):
  for text in data['Text']:
    for word in text:
      if word not in term_freq_data:
        term_freq_data[word]={}
      if cat_type not in term_freq_data[word]:
        term_freq_data[word][cat_type] = 0
      term_freq_data[word][cat_type]+=1
  return  term_freq_data  



In [459]:
# wrapper function to calculate term frequency 
def calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain):
  term_freq = cal_tf(term_freq, cat_business, "business")
  term_freq = cal_tf(term_freq, cat_tech, "tech")
  term_freq = cal_tf(term_freq, cat_politics, "politics")
  term_freq = cal_tf(term_freq, cat_sport, "sport")
  term_freq = cal_tf(term_freq, cat_entertain, "entertainment")
  return term_freq

In [460]:

term_freq = calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)
# print(term_freq)

In [461]:
#function to calculate class frequency
def cal_cf(term_freq,class_freq,data,cat_type):
  
  words = term_freq.keys()
  for word in words:
      if word not in class_freq:
        class_freq[word]=0
      if cat_type in term_freq[word]:
        class_freq[word]+=1
  return  class_freq  



In [462]:
# wrapper function to calculate class frequency
def calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain):
  class_freq = cal_cf(term_freq, class_freq, cat_business, "business")
  class_freq = cal_cf(term_freq, class_freq, cat_tech, "tech")
  class_freq = cal_cf(term_freq, class_freq, cat_politics, "politics")
  class_freq = cal_cf(term_freq, class_freq, cat_sport, "sport")
  class_freq = cal_cf(term_freq, class_freq, cat_entertain, "entertainment")
  return class_freq


In [463]:
class_freq = calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)
# print(class_freq)

In [464]:
#function to calculate tf-icf score
def tf_icf(term_freq,class_freq):
  tf_icf_dict = {}
  for word in term_freq.keys():
      tf_icf_dict[word] = {}
      for category in categories:
          tf = term_freq[word].get(category, 0)
          
          cf = class_freq[word]
          icf = np.log10(num_class / cf)
        
          tf_icf_dict[word][category] = tf * icf
  return tf_icf_dict

In [465]:

tf_icf_dict = tf_icf(term_freq,class_freq)

In [466]:
df = pd.DataFrame(tf_icf_dict) # converting dictionary of tf-icf into dataframe


In [ ]:
# creating dataframe to map the tf-icf values of each word to the words based on their occurance in each sentence
empty_dataframe = []
df1 = pd.DataFrame(empty_dataframe,columns = tf_icf_dict.keys(),index = data_train.index )


In [468]:
# mapping the tf-icf values to the words in the respected sentences
def fill_data(df, df1, data_train):
  ind = data_train.index
  for i in range(len(ind)):
    cat = data_train.iloc[i]['Category']
    text = data_train.iloc[i]['Text']
 
    for word in text:
   
      df1.iloc[i][word] = df.loc[cat][word]
  
  return df1

In [469]:
df1 = fill_data(df, df1, data_train).copy()

In [470]:
df1 = df1.fillna(0) # replacing nan values with 0

In [471]:
# diving the data into features and label
x = df1
y = data_train['Category']


**Dataset**

In [472]:
# function to split the dataset into train and test
def training_testing_split(x,y,test_size):
  train_size = int(len(x)*(1-test_size))
  train_data_x = x[:train_size]
  train_data_y = y[:train_size]
  test_data_x = x[train_size:]
  test_data_y = y[train_size:]

  return train_data_x, test_data_x, train_data_y, test_data_y

In [473]:
# splitting the datadset into the ratio of 70:30
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

In [474]:
x_train

,worldcom,ex,bos,launch,defence,lawyer,defend,former,chief,bernie,...,103,playboyz,maniac,granada,cheeky,cassette,grandma,verbalicious,stereophonics,rapport
0,37.74438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.59176,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1038,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1039,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1040,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1041,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Training the Naive Bayes

In [475]:
from sklearn.naive_bayes import MultinomialNB

In [476]:
# wrapper function to implement Naive Bayes with data having tf-icf weighting scheme
def naive_bayes(x_train,y_train,x_test):
  mb = MultinomialNB()
  mb.fit(x_train, y_train)
  
# making predictions on the testing set
  y_pred = mb.predict(x_test)
  return y_pred

In [477]:
# calculating the probability of each category
def calc_freq_cat(y_train):
  freq_cat = dict()
  prob = dict()
  num_docs  = len(y_train)

  for cat in y_train:
    if cat not in freq_cat:
      freq_cat[cat]=0
    freq_cat[cat]+=1

# print(freq_cat)

  for cat, freq in freq_cat.items():
    prob[cat] = freq/num_docs

  print(prob)



In [504]:
# calling the function to sgow the category wise probability
calc_freq_cat(y_train)

{'business': 0.22722914669223393, 'tech': 0.17833173537871524, 'politics': 0.18024928092042186, 'sport': 0.23873441994247363, 'entertainment': 0.17545541706615533}


**Testing the Naive Bayes**

In [478]:
lst_y_test = list(y_test)
# using testing set to evaluate the classifier
y_pred = naive_bayes(x_train,y_train,x_test)

In [482]:
y_test_df = list(y_test)

In [484]:
#comparing testing and predicted labels
comp_test_pred = pd.DataFrame(y_test_df,columns=["Actual Label"])
comp_test_pred.insert(1,"Predicted Label",y_pred,True)
print(comp_test_pred)


      Actual Label Predicted Label
0             tech            tech
1         politics        politics
2            sport           sport
3         politics        politics
4             tech            tech
..             ...             ...
442  entertainment   entertainment
443  entertainment   entertainment
444       business        business
445           tech            tech
446           tech            tech

[447 rows x 2 columns]


In [485]:
# function to calculate accuracy
def calc_acc(y_test, y_pred):
  acc = [1 if y_test[i]==y_pred[i] else 0 for i in range(len(y_test))]

  return np.sum(acc)/len(y_test)

In [487]:
# y_pred = naive_bayes(x_train,y_train,x_test)

In [488]:
# calculation confusion matrix for using it in the calculation of precision, recall
from sklearn.metrics import confusion_matrix
conf_mat = confusion_matrix(lst_y_test,y_pred)


In [489]:
# function to calculate tp, tn, fp, fn
def calc_tp_tf_fp_fn(conf_mat):
  tp = np.diag(conf_mat)
  fp = np.sum(conf_mat,axis = 0) - tp
  fn = np.sum(conf_mat,axis = 1) - tp
  tn = np.sum(conf_mat) - tp - fp - fn

  return tp, fp, fn, tn

In [490]:
# function to calculate precision
def calc_prec(tp, fp, fn, tn):
  prec = np.sum(tp)/np.sum(tp+fp)
  return prec

In [491]:
# function to calculate recall
def calc_rec(tp, fp, fn, tn):
  rec = np.sum(tp)/np.sum(tp+fn)
  return rec

In [492]:
# printing the evaluation metrics with lemmatization and 70:30 split ratio
tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
accuracy = calc_acc(lst_y_test,y_pred)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec) # calculating f1 score


print("Accuracy: ",accuracy)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)


Accuracy:  0.9977628635346756
Precision:  0.9977628635346756
Recall:  0.9977628635346756
F1 Score:  0.9977628635346756


**Improving the classifier**

In [499]:
# data input for improving the classifier
data_train = data_train_temp.copy()

# lowering the case of the dataset
data_train['Category'] = data_train['Category'].str.lower()
data_train['Text'] = data_train['Text'].str.lower()
data_train.head()


,Text,Category
0,worldcom ex-boss launches defence lawyers defe...,business
1,german business confidence slides german busin...,business
2,bbc poll indicates economic gloom citizens in ...,business
3,lifestyle governs mobile choice faster bett...,tech
4,enron bosses in $168m payout eighteen former e...,business


In [500]:

categories = data_train['Category'].unique()

term_freq = {}
class_freq = {}
num_class = len(categories)

  

In [501]:
# applying other preprocessing along with stemming in place of lemmatization
data_train['Text'] = data_train['Text'].apply(removePunctuation)

data_train['Text'] = data_train['Text'].apply(removeWords)


data_train['Text'] = data_train['Text'].apply(stemming)


cat_business, cat_tech, cat_politics, cat_sport, cat_entertain = get_cat(categories,data_train)

term_freq = calc_term_freq(term_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)

class_freq = calc_class_freq(term_freq,class_freq,cat_business,cat_tech,cat_politics,cat_sport,cat_entertain)


In [502]:
#calculating tf-icf and repeating the above steps
tf_icf_dict = tf_icf(term_freq,class_freq)

df = pd.DataFrame(tf_icf_dict)


empty_dataframe = []
df1 = pd.DataFrame(empty_dataframe,columns = tf_icf_dict.keys(),index = data_train.index )

df1 = fill_data(df, df1, data_train).copy()
df1 = df1.fillna(0)




In [505]:
# spliting the data into features and label
x = df1
y = data_train['Category']


In [507]:
# splitting the data into 70:30 splits with stemming to compare with previous result with lemmatization
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)




conf_mat = confusion_matrix(lst_y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)

accuracy = calc_acc(lst_y_test,y_pred)

prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Accuracy: ",accuracy)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.9977628635346756
Precision:  0.9977628635346756
Recall:  0.9977628635346756
F1 Score:  0.9977628635346756


In [508]:
# splitting the data with 60:40 ratio
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.4)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)




conf_mat = confusion_matrix(lst_y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)

accuracy = calc_acc(lst_y_test,y_pred)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)

print("Accuracy: ",accuracy)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.9983221476510067
Precision:  0.9983221476510067
Recall:  0.9983221476510067
F1 Score:  0.9983221476510067


In [509]:
# splitting the data with 50:50 ratio
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.5)

lst_y_test = list(y_test)

y_pred = naive_bayes(x_train, y_train,x_test)

accuracy = calc_acc(lst_y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(lst_y_test,y_pred)
  #print(conf_mat)

tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.9986577181208054
Precision:  0.9986577181208054
Recall:  0.9986577181208054
F1 Score:  0.9986577181208054


In [436]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [510]:
#applying tf-idf to the data for comparison with tf- with 70:30 split ratio
x = data_train['Text']
y = data_train['Category']
x_train, x_test, y_train, y_test = training_testing_split(x,y,test_size=0.3)

# lst_y_test = list(y_test)

In [511]:
vect = TfidfVectorizer()
# converting the list to string
x_train_vect = [' '.join(data) for data in x_train]
x_test_vect = [' '.join(data) for data in x_test]


In [512]:
x_train_vect = vect.fit_transform(x_train_vect)
x_test_vect = vect.transform(x_test_vect)

In [513]:
#applying naive bayes
mb = MultinomialNB()
mb.fit(x_train_vect, y_train)
y_test = list(y_test)
y_pred = mb.predict(x_test_vect)

In [514]:
# evaluation metrices
accuracy = calc_acc(y_test,y_pred)
print("Accuracy: ",accuracy)


conf_mat = confusion_matrix(y_test,y_pred)


tp, fp, fn, tn = calc_tp_tf_fp_fn(conf_mat)
prec = calc_prec(tp, fp, fn, tn)
rec = calc_rec(tp, fp, fn, tn)
f1_score = 2*prec*rec/(prec+rec)
print("Precision: ",prec)
print("Recall: ",rec)
print("F1 Score: ", f1_score)

Accuracy:  0.970917225950783
Precision:  0.970917225950783
Recall:  0.970917225950783
F1 Score:  0.9709172259507831
